# Imports

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import warnings 

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../data/USD-EGP Time series Data - raw.csv")
df.head()

,Date,USD,Date.1,EUR,Date.2,GBP
0,12/01/2003 23:58:00,6.1070,12/01/2003 23:58:00,7.304583,12/01/2003 23:58:00,10.494269
1,12/02/2003 23:58:00,6.0749,12/02/2003 23:58:00,7.340909,12/02/2003 23:58:00,10.505325
2,12/03/2003 23:58:00,6.1057,12/03/2003 23:58:00,7.398887,12/03/2003 23:58:00,10.545154
3,12/04/2003 23:58:00,6.1161,12/04/2003 23:58:00,7.384579,12/04/2003 23:58:00,10.519692
4,12/05/2003 23:58:00,6.1088,12/05/2003 23:58:00,7.441740,12/05/2003 23:58:00,10.580442


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6241 entries, 0 to 6240
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    6241 non-null   object 
 1   USD     6241 non-null   float64
 2   Date.1  6239 non-null   object 
 3   EUR     6239 non-null   float64
 4   Date.2  6239 non-null   object 
 5   GBP     6239 non-null   float64
dtypes: float64(3), object(3)
memory usage: 292.7+ KB


# Data cleaning

In [4]:
df["Date"] = pd.to_datetime(df['Date'], format='%m/%d/%Y %H:%M:%S')
df["Date.1"] = pd.to_datetime(df['Date.1'], format='%m/%d/%Y %H:%M:%S')
df["Date.2"] = pd.to_datetime(df['Date.2'], format='%m/%d/%Y %H:%M:%S')

df.head()

,Date,USD,Date.1,EUR,Date.2,GBP
0,2003-12-01 23:58:00,6.1070,2003-12-01 23:58:00,7.304583,2003-12-01 23:58:00,10.494269
1,2003-12-02 23:58:00,6.0749,2003-12-02 23:58:00,7.340909,2003-12-02 23:58:00,10.505325
2,2003-12-03 23:58:00,6.1057,2003-12-03 23:58:00,7.398887,2003-12-03 23:58:00,10.545154
3,2003-12-04 23:58:00,6.1161,2003-12-04 23:58:00,7.384579,2003-12-04 23:58:00,10.519692
4,2003-12-05 23:58:00,6.1088,2003-12-05 23:58:00,7.441740,2003-12-05 23:58:00,10.580442


#### some dates dont match on all currencies

In [5]:
n1 = set(df["Date"])
n2 = set(df["Date.1"])
n3 = set(df["Date.2"])

uncommon = {*(n1^n2), *(n1^n3) , *(n2^n3)}
common = list(set(df["Date"]) & set(df["Date.1"]) & set(df["Date.2"]))

print("uncommon: ",len(uncommon), "common: ",len(common))

uncommon:  3 common:  6239


In [6]:
df_clean = pd.DataFrame({"Date": df["Date"],"USD":None,"EUR":None,"GBP":None})
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6241 entries, 0 to 6240
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    6241 non-null   datetime64[ns]
 1   USD     0 non-null      object        
 2   EUR     0 non-null      object        
 3   GBP     0 non-null      object        
dtypes: datetime64[ns](1), object(3)
memory usage: 195.2+ KB


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6241 entries, 0 to 6240
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    6241 non-null   datetime64[ns]
 1   USD     6241 non-null   float64       
 2   Date.1  6239 non-null   datetime64[ns]
 3   EUR     6239 non-null   float64       
 4   Date.2  6239 non-null   datetime64[ns]
 5   GBP     6239 non-null   float64       
dtypes: datetime64[ns](3), float64(3)
memory usage: 292.7 KB


#### reorder each currency row by its correct date
else set it as null

In [8]:
def cln1(df,date,st,dt):
    for i,g in zip(df[dt].values,df[st].values):
        if i == date:
            return g
    return None

In [9]:
for i in range(df.shape[0]):
    
    x = df["Date"][i]
    
    us =  cln1(df,x,"USD","Date") 
    eu = cln1(df,x,"EUR","Date.1") 
    gb = cln1(df,x,"GBP","Date.2")
    
    df_clean['USD'][i] = us
    df_clean['EUR'][i] = eu
    df_clean['GBP'][i] = gb
    
df_clean

,Date,USD,EUR,GBP
0,2003-12-01 23:58:00,6.107,7.304583,10.494269
1,2003-12-02 23:58:00,6.0749,7.340909,10.505325
2,2003-12-03 23:58:00,6.1057,7.398887,10.545154
3,2003-12-04 23:58:00,6.1161,7.384579,10.519692
4,2003-12-05 23:58:00,6.1088,7.44174,10.580442
...,...,...,...,...
6236,2023-03-27 23:58:00,30.8991,33.394,37.99609
6237,2023-03-28 23:58:00,30.9085,33.491,38.10293
6238,2023-03-29 23:58:00,30.8019,33.4,37.92459
6239,2023-03-30 23:58:00,30.8944,33.69,38.27213


In [10]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6241 entries, 0 to 6240
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    6241 non-null   datetime64[ns]
 1   USD     6241 non-null   object        
 2   EUR     6239 non-null   object        
 3   GBP     6239 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 195.2+ KB


#### replace each null value 
by the average of the cell before it and after it.<br>if null it would take the cell before that or after that.

In [11]:
def cln2(df,col):
    for i in range(df_clean.shape[0]):
        if df_clean[col][i] == None:
            
            x1 = df_clean[col][i]
            x2 = df_clean[col][i]
            c1 = 1
            c2 = 1
            
            while(x2 == None):
                x2= df_clean[col][i+c2]
                c2 = c2+1  
                
            while(x1 == None):
                x1= df_clean[col][i-c1]
                c1 = c1+1
            df_clean[col][i] = (x1+x2)/2

In [12]:
cln2(df_clean,"GBP")
cln2(df_clean,"EUR")

In [13]:
df_clean.set_index("Date",inplace= True)

In [14]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6241 entries, 2003-12-01 23:58:00 to 2023-03-31 23:58:00
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   USD     6241 non-null   object
 1   EUR     6241 non-null   object
 2   GBP     6241 non-null   object
dtypes: object(3)
memory usage: 195.0+ KB


#### Save and reuse 

In [15]:
df_clean.to_csv("../data/USD-EUR-GBP to EGP Time series Data clean.csv")